In [165]:
from twelvedata import TDClient
import os
import pandas as pd

In [166]:
td = TDClient(os.getenv('TD_API'))

In [167]:
all_symbols = ("AUD/CAD","AUD/CHF","AUD/JPY","AUD/NZD","AUD/USD",
               "CAD/CHF", "CAD/JPY", "CHF/JPY", 
               "EUR/AUD", "EUR/CAD", "EUR/CHF", "EUR/GBP", "EUR/JPY", "EUR/NZD", "EUR/USD",
               "GBP/AUD", "GBP/CAD", "GBP/CHF", "GBP/JPY", "GBP/NZD", "GBP/USD",
               "NZD/CAD", "NZD/CHF", "NZD/JPY", "NZD/USD", 
               "USD/CAD", "USD/CHF", "USD/JPY")

In [168]:
portfolio_sheet = pd.read_csv('portfolio_example.csv')

In [169]:
for pair in portfolio_sheet['pair']:
    if pair in all_symbols:
      continue
    else:
      print("Symbol is not allowed or mispelled.  Please check the 'all_symbols' variable.")
      exit()

In [170]:
table = pd.DataFrame()

for _symbol in portfolio_sheet['pair']:
  ts = td.time_series(
    symbol = _symbol,
    interval="1day",
    outputsize="14"
  ).as_pandas()

  ts.drop(['open', 'high', 'low'], axis=1, inplace=True)
  ts.rename(columns={'close': str(_symbol)}, inplace=True)
  table = pd.concat([table, ts], axis=1)

print(table.corr())


          GBP/USD   EUR/USD   USD/JPY   AUD/CHF   AUD/NZD
GBP/USD  1.000000  0.489973 -0.318275 -0.451443 -0.651622
EUR/USD  0.489973  1.000000 -0.684288  0.345194 -0.140338
USD/JPY -0.318275 -0.684288  1.000000 -0.540898  0.120083
AUD/CHF -0.451443  0.345194 -0.540898  1.000000  0.581961
AUD/NZD -0.651622 -0.140338  0.120083  0.581961  1.000000


In [171]:
pivot_risk = portfolio_sheet.iloc[0,1]
drawdown = pivot_risk

In [172]:
pair_counter = 1
for i in range(((len(portfolio_sheet) -1))):
  corr = table.iloc[:,0].corr(table.iloc[:,pair_counter])
  pair_risk = portfolio_sheet.iloc[pair_counter, 1]
  add_drawdown = corr * pair_risk
  drawdown += add_drawdown
  if pair_counter < len(portfolio_sheet) -1:
    pair_counter += 1

print("Expected drawdown is: %", -1 * round(drawdown, 2))

Expected drawdown is: % -1.07
